In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [2]:
# define import path
path_import = Path("../data/outputs/03_example_CCTS_N-1_03_CT_infisbal")

# define export path
path_export = Path("../data/03_example_CCTS_N-1_03_CT_infisbal_adjusted/energy_system/carbon_emissions_annual_limit_adjustment.csv")

In [3]:
# infeasible solution
results = Results(path_import)

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/extensions/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins.extensions')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').decla

In [4]:
# get flow export and save as csv
results.get_df('flow_export', to_csv="flow_export")

carrier      node              failure_states                              time_operation
co2_ambient  swiss_storage_CH  no_failure_technology: no_failure_location  0                 0.0
                                                                           1                 0.0
                                                                           2                 0.0
                                                                           3                 0.0
                                                                           4                 0.0
                                                                                            ... 
co2_stored   waste_2_CH        flowline: waste_2_CH-swiss_storage_CH       0                 0.0
                                                                           1                 0.0
                                                                           2                 0.0
                                     

In [5]:
# get flow export csv
df = pd.read_csv('flow_export.csv')

In [6]:
# find unique failure states and time steps
states = df['failure_states'].unique()
time_steps = df['time_operation'].unique()

# filter for co2 stored
co2_stored = df[df['carrier'] == "co2_stored"]

# Initialize an empty list to store data
table_data = []

# Iterate over time_steps
for time_step in range(len(time_steps)):
    co2_stored_time_step = co2_stored[co2_stored['time_operation'] == time_step]
    co2_stored_no_failure = co2_stored_time_step[co2_stored_time_step['failure_states'] == "no_failure_technology: no_failure_location"].sum()
    co2_not_stored = (len(states) * co2_stored_no_failure["flow_export"] - co2_stored_time_step.sum()["flow_export"]) * 8.76
    
    # Append data as a tuple to the list
    table_data.append((time_steps[time_step], co2_not_stored))

# Create a DataFrame from the list
table_df = pd.DataFrame(table_data, columns=["year", "carbon_emissions_limit"])
table_df['year'] = table_df['year'] + 2025

# safe the table DataFrame
filepath = Path(path_export)
table_df.to_csv(filepath, index=False)

OSError: Cannot save file into a non-existent directory: '../data/03_example_CCTS_N-1_03_CT_infisbal_adjusted/energy_system'